# INFORMACIÓN DEL PROYECTO

### TÍTULO

Titulo = "El quién, el qué y el cuánto del consumo cultural en Cataluña"

### TEMA

Tema = "Predicción de tipos y volúmenes de consumo cultural en función de los perfiles de usaurio en Cataluña, según las Encuestas de Participación Cultural de la Generalitat de los años 2018 a 2023"

### OBJETIVO

Define aquí lo que serán los objetivos de tu proyecto.

Objetivo 1 = Predecir qué y cuánta cultura consumirá un ciudadano X dado su perfil (X ciudadano, que es /tiene/ hace/ viene de/ piensa X, etc, visitará x veces un museo / cine/etc.)<br>
Objetivo 2 = Clasificar los hábitos de consumo cultural según variables tradicionales / demográficas (qué consumen los jóvenes, las mujeres de más de 60, los adultos de zonas rurales).<br>
Objetivo 3 = Encontrar perfiles de usuarios de cultura en función de los hábitos y gustos, no de la demografía a partir de patrones de consumo común y mediante clustering, y descubrir qué variables resultan más relevantes<br>

POSIBLEMENTE NO: (Demasiado trabajo extra y no tanta relevancia para este proyecto, aunque a mi me parezca interesante)
Objetivo 4 = Hacer un clustering a partir de los datos de 1028 siguiendo el mismo criterio que con los datos de 2018, y analizar en qué medida hay o no cambios.<br>

## OBTENCIÓN DE LOS DATOS

### DATASETS Y FUENTES ALTERNATIVAS DE DATOS

Incluye aquí una vista del dataset o datasets de los que partirás para poder evaluar tu hipótesis. <br>
También incluye el origen de estos datos y su fuente.

Enquesta de participació cultural de la Generalitat de Catalunya (2018, 2019, 2020, 2021, 2022, 2023) 

Lo más probable es que solo trabaje con los datos de 2023, y no con la evolución de las prácticas culturales a través de losaños.

https://analisi.transparenciacatalunya.cat/ca/browse?q=enquesta%20participacio%20cultural

In [1]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [2]:
epcc23 = pd.read_excel('./data/BBDD_EPCC_23.xlsx')

In [3]:
epcc23.head()

,ID,Elevació,Mètode,SEXE,EDAT,AmbitTerritorial,GrandariaMunicipi,I1,I2EA,I2EB,...,CS6,CS11,CS12,CS13,CS15A,CS15,CS21,G1,G2,G2A
0,100,674.941909,CAWI,H,14-19,METROPOLITÀ,MÉS DE 50.000 - RESTA CATALUNYA,2,1,15,...,NaN,2.0,NaN,3,2,1.0,NaN,1,7.0,NaN
1,201,674.941909,CAWI,H,14-19,METROPOLITÀ,MÉS DE 50.000 - RESTA CATALUNYA,2,1,16,...,NaN,2.0,NaN,2,3,1.0,NaN,2,99.0,NaN
2,202,674.941909,CATI,H,14-19,METROPOLITÀ,MÉS DE 50.000 - RESTA CATALUNYA,1,1,16,...,NaN,2.0,NaN,2,3,1.0,NaN,2,99.0,NaN
3,303,674.941909,CAWI,H,14-19,METROPOLITÀ,MÉS DE 50.000 - RESTA CATALUNYA,1,1,16,...,NaN,2.0,NaN,2,3,1.0,NaN,2,5.0,NaN
4,501,674.941909,CAWI,H,14-19,METROPOLITÀ,MÉS DE 50.000 - RESTA CATALUNYA,1,1,16,...,NaN,2.0,NaN,2,3,1.0,NaN,2,NaN,NaN


#### Diccionario de códigos

In [4]:
dicc_codis = pd.read_excel("./data/diccionari_codis_23.xlsx")

In [5]:
dicc_codis.head()

,Valors de les variables,Unnamed: 1,Unnamed: 2
0,Valor,NaN,Etiqueta
1,I1,0.0,Ningú de la llar parla cap d’aquestes llengües
2,NaN,1.0,Català
3,NaN,2.0,Castellà
4,I2EA,1.0,Home


#### Diccionario de variables

In [6]:
dicc_variables = pd.read_excel("./data/diccionari_variables_23.xlsx")

In [7]:
dicc_variables.head()

,Informació de les variables,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Variable,Posición,Etiqueta,Nivel de medición,Rol,Ancho de columna,Alineación,Formato de impresión,Formato de grabación
1,ID,1,Identificador,Nominal,Entrada,5,Derecha,F8,F8
2,Elevació,2,Elevació,Escala,Entrada,5,Derecha,F11.6,F11.6
3,Mètode,3,Mètode,Nominal,Entrada,5,Izquierda,A4,A4
4,ORDRE,4,Ordre,Nominal,Entrada,5,Izquierda,A2,A2


-------------------------------------

### Limpieza de datos

In [8]:
epcc23co = epcc23.copy()

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [10]:
epcc23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4124 entries, 0 to 4123
Columns: 519 entries, ID to G2A
dtypes: float64(447), int64(46), object(26)
memory usage: 16.3+ MB


##### Primero voy a trabajar las columnas que tienen strings para convertirlas en numéricas

##### Empiezo con los binarios

In [11]:
print(epcc23['Mètode'].unique())

['CAWI' 'CATI']


In [12]:
epcc23['Mètode'] = epcc23['Mètode'].map({'CAWI': 0, 'CATI': 1})

In [13]:
print(epcc23['SEXE'].unique())

['H' 'D']


In [14]:
epcc23['SEXE'] = epcc23['SEXE'].map({'H': 0, 'D': 1})

##### Ahora los ordinales

In [15]:
print(epcc23['EDAT'].unique())


['14-19' '20-24' '35-54' '25-34' '>=55']


In [16]:
# Convertir la columna EDAT a una escala
age_mapping = {
    '14-19': 0,
    '20-24': 1,
    '25-34': 2,
    '35-54': 3,
    '>=55': 4
}
epcc23['EDAT'] = epcc23['EDAT'].map(age_mapping)


##### Y ahora las demás que quedan para One hot encoding

In [17]:
print(epcc23['AmbitTerritorial'].unique())


['METROPOLITÀ' 'PENEDÈS' 'COMARQUES CENTRALS' 'COMARQUES GIRONINES'
 'CAMP DE TARRAGONA' 'ALT PIRINEU I ARAN' "TERRES DE L'EBRE" 'PONENT']


In [18]:
# Convertir la columna AmbitTerritorial a one-hot encoding
epcc23 = pd.get_dummies(epcc23, columns=['AmbitTerritorial'])


In [19]:
print(epcc23['GrandariaMunicipi'].unique())


['MÉS DE 50.000 - RESTA CATALUNYA' 'BARCELONA CIUTAT' 'DE 10.001 A 50.000'
 'DE 2.001 A 10.000' 'FINS A 2.000']


In [20]:
epcc23 = pd.get_dummies(epcc23, columns=['GrandariaMunicipi'])


In [21]:
# A ver cuántos NaN tengo:
nan_percentage = epcc23.isna().mean() * 100

# Crear un DataFrame con la información
nan_info = pd.DataFrame({
    'Column': nan_percentage.index,
    'NaN_Percentage': nan_percentage.values
})

# Mostrar el DataFrame ordenado por el porcentaje de NaNs
nan_info_sorted = nan_info.sort_values(by='NaN_Percentage', ascending=False)
print(nan_info_sorted)

                                                Column  NaN_Percentage
195                                            VJO8#95           100.0
196                                            VJO8#99           100.0
176                                            VJO6#99           100.0
241                                               M7#9           100.0
477                                            LN7#2#4           100.0
..                                                 ...             ...
372                                                LL1             0.0
335                                                EX1             0.0
312                                                ES1             0.0
278                                                CN1             0.0
529  GrandariaMunicipi_MÉS DE 50.000 - RESTA CATALUNYA             0.0

[530 rows x 2 columns]


In [22]:
nan_info_nonzero = nan_info_sorted[nan_info_sorted['NaN_Percentage'] > 0]
print(nan_info_nonzero)


        Column  NaN_Percentage
195    VJO8#95      100.000000
196    VJO8#99      100.000000
176    VJO6#99      100.000000
241       M7#9      100.000000
477    LN7#2#4      100.000000
..         ...             ...
69   CA6#value        0.193986
474    LN7#2#1        0.169738
30   PP3#value        0.169738
42    R3#value        0.169738
20    D3#value        0.048497

[463 rows x 2 columns]


#### Antes de procesar los nulos, vamos a trabajar los grupos de columnas codificadas

In [23]:
# Agrupar las encoded columns listas explícitas (las que son el resultado de un one hot encoding que representan diferentes opciones de respuesta a una misma pregunta)

encoded_groups = [
    ["D2#1", "D2#2", "D2#4", "D2#99"],
    ["D3#1", "D3#2","D3#3", "D3#99"],
    ["PP0#1", "PP0#2", "PP0#3", "PP0#99"],
    ["PP3#1","PP3#2","PP3#3","PP3#99"],
    ["R2#1", "R2#2", "R2#3", "R2#4", "R2#5", "R2#99"],
    ["R3#1", "R3#2", "R3#3", "R3#99"],
    ["CA2A#1","CA2A#2", "CA2A#3", "CA2A#4", "CA2A#5", "CA2A#6", "CA2A#99"],
    ["T4#2", "T4#3", "T4#4", "T4#5", "T4#6", "T4#7", "T4#8", "T4#11", "T4#99"],
    ["CA6#1", "CA6#2", "CA6#3", "CA6#4", "CA6#5", "CA6#6", "CA6#99"],
    ["CA4#2", "CA4#1", "CA4#3", "CA4#4", "CA4#99"],
    ["CA5#1", "CA5#2", "CA5#3", "CA5#4", "CA5#5", "CA5#6", "CA5#99"],
    ["IT4#1","IT4#2", "IT4#3", "IT4#4", "IT4#5", "IT4#6", "IT4#99"],
    ["IT6#1", "IT6#2", "IT6#3", "IT6#4","IT6#5", "IT6#6", "IT6#7", "IT6#8", "IT6#9", "IT6#10", "IT6#99"],
    ["VJO1#1", "VJO1#2", "VJO1#3", "VJO1#4", "VJO1#5", "VJO1#6", "VJO1#7", "VJO1#8", "VJO1#9", "VJO1#10", "VJO1#11", "VJO1#99"],
    ["VJ2#1", "VJ2#2", "VJ2#3", "VJ2#4", "VJ2#5", "VJ2#6", "VJ2#7", "VJ2#8", "VJ2#9", "VJ2#10", "VJ2#99"],
    ["VJ3#1", "VJ3#2", "VJ3#3", "VJ3#4", "VJ3#5", "VJ3#6", "VJ3#7", "VJ3#8", "VJ3#99"],
    ["VJ6#1", "VJ6#2", "VJ6#3", "VJ6#4", "VJ6#5", "VJ6#6", "VJ6#7", "VJ6#99"],
    ["VJ7#1", "VJ7#2", "VJ7#3", "VJ7#4", "VJ7#5", "VJ7#6", "VJ7#7", "VJ7#8", "VJ7#9", "VJ7#10", "VJ7#11", "VJ7#99"],
    ["VJO6#1", "VJO6#2", "VJO6#3", "VJO6#4", "VJO6#5", "VJO6#6", "VJO6#7", "VJO6#95", "VJO6#99"],
    ["VJO7A#1", "VJO7A#2", "VJO7A#99"],
    ["VJO8#1", "VJO8#2", "VJO8#3", "VJO8#4", "VJO8#5", "VJO8#6", "VJO8#7", "VJO8#8", "VJO8#9", "VJO8#95", "VJO8#99"],
    ["M3#1", "M3#2", "M3#3", "M3#4", "M3#5", "M3#6", "M3#7", "M3#99"],
    ["M4#1", "M4#7", "M4#9", "M4#10", "M4#11", "M4#13", "M4#18", "M4#19", "M4#99"],
    ["M5#1", "M5#2", "M5#3", "M5#4", "M5#5", "M5#6", "M5#7", "M5#8", "M5#9", "M5#99"],
    ["M7#1", "M7#2", "M7#3", "M7#4", "M7#5", "M7#6", "M7#7", "M7#8", "M7#9", "M7#10", "M7#11", "M7#99"],
    ["CO3#1", "CO3#2", "CO3#3", "CO3#4", "CO3#5", "CO3#6", "CO3#7", "CO3#99"],
    ["CO7#1", "CO7#2", "CO7#3", "CO7#4", "CO7#5", "CO7#6", "CO7#7", "CO7#8", "CO7#9", "CO7#10", "CO7#99"],
    ["CO9#1", "CO9#7", "CO9#9", "CO9#10", "CO9#11", "CO9#13", "CO9#18", "CO9#19", "CO9#99"],
    ["CN2#1", "CN2#2", "CN2#3", "CN2#4", "CN2#5", "CN2#6", "CN2#99"],
    ["CN5#1", "CN5#2", "CN5#3", "CN5#4", "CN5#5", "CN5#6", "CN5#9", "CN5#10", "CN5#11", "CN5#12", "CN5#13", "CN5#99"],
    ["CN6#1", "CN6#2", "CN6#3", "CN6#4", "CN6#5", "CN6#6", "CN6#7", "CN6#8", "CN6#9", "CN6#88", "CN6#99"],
    ["ES3#1", "ES3#2", "ES3#3", "ES3#4", "ES3#5", "ES3#6", "ES3#7", "ES3#99"],
    ["ES6#1", "ES6#2", "ES6#3", "ES6#4", "ES6#5", "ES6#6", "ES6#7", "ES6#8", "ES6#9", "ES6#88", "ES6#99"],
    ["EX7#1", "EX7#2", "EX7#3", "EX7#4", "EX7#5", "EX7#6", "EX7#7", "EX7#8", "EX7#9", "EX7#88", "EX7#99"],
    ["EX9#1", "EX9#2", "EX9#3", "EX9#4", "EX9#5", "EX9#6", "EX9#99"],
    ["EX13#1", "EX13#2", "EX13#3", "EX13#4", "EX13#7", "EX13#10", "EX13#11", "EX13#12", "EX13#99"],
    ["LL5#2", "LL5#3", "LL5#4", "LL5#5", "LL5#6", "LL5#7", "LL5#8", "LL5#9", "LL5#10", "LL5#11", "LL5#99"],
    ["LL5A#1", "LL5A#2", "LL5A#3", "LL5A#4", "LL5A#5", "LL5A#6", "LL5A#7", "LL5A#8", "LL5A#9", "LL5A#10", "LL5A#99"],
    ["LL11#2", "LL11#3", "LL11#4", "LL11#7", "LL11#9", "LL11#11", "LL11#12", "LL11#13", "LL11#16", "LL11#99"],
    ["BI2#1", "BI2#2", "BI2#3", "BI2#4", "BI2#99"],
    ["BI3#1", "BI3#2", "BI3#3", "BI3#5", "BI3#6", "BI3#7", "BI3#8", "BI3#10", "BI3#99"],
    ["AS2#1", "AS2#2", "AS2#3", "AS2#4", "AS2#5", "AS2#6", "AS2#7", "AS2#8", "AS2#9", "AS2#10", "AS2#11", "AS2#16", "AS2#17", "AS2#99"],
    ["LN2#1", "LN2#2", "LN2#3", "LN2#4", "LN2#5", "LN2#6", "LN2#99"],
    ["LN7#1#1", "LN7#1#2", "LN7#1#3", "LN7#1#4", "LN7#1#99"],
    ["LN7#2#1", "LN7#2#2", "LN7#2#3", "LN7#2#4", "LN7#2#99"],
    ["LN7#3#1", "LN7#3#2", "LN7#3#3", "LN7#3#4", "LN7#3#99"],
    ["LN7#4#0", "LN7#4#1", "LN7#4#2", "LN7#4#3", "LN7#4#9"],
]

In [24]:
modified_cells_count = 0

In [25]:
# Me aseguro de que, dentro de cada grupo de preguntas, si no hay ninguna opción marcada, que se marque la de NS/NC, 
# que es la correspondiente a la columna acabada en #99 dentro de dicho grupo

for group in encoded_groups:
    # Verifica que las columnas existan en el DataFrame
    valid_columns = [col for col in group if col in epcc23.columns]
    if valid_columns:
        non_ns_nc_cols = [col for col in valid_columns if not col.endswith('#99')]
        ns_nc_col = [col for col in valid_columns if col.endswith('#99')]
        
        if ns_nc_col:
            ns_nc_col = ns_nc_col[0]
            
            # Si ninguna columna tiene valor, poner 1 en la columna de NS/NC
            ns_nc_updates = epcc23[non_ns_nc_cols].sum(axis=1) == 0
            epcc23[ns_nc_col] = np.where(ns_nc_updates, 1, epcc23[ns_nc_col])
            modified_cells_count += ns_nc_updates.sum()
            
            # Rellenar con 0 las columnas que no tengan ningún valor dentro del grupo
            epcc23[valid_columns] = epcc23[valid_columns].fillna(0)

# Mostrar el total de casillas modificadas
print(f"Total de casillas modificadas en grupos NS/NC: {modified_cells_count}")


Total de casillas modificadas en grupos NS/NC: 75608


In [26]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
epcc23.head()

,ID,Elevació,Mètode,SEXE,EDAT,I1,I2EA,I2EB,AJ1,I4,...,AmbitTerritorial_COMARQUES GIRONINES,AmbitTerritorial_METROPOLITÀ,AmbitTerritorial_PENEDÈS,AmbitTerritorial_PONENT,AmbitTerritorial_TERRES DE L'EBRE,GrandariaMunicipi_BARCELONA CIUTAT,GrandariaMunicipi_DE 10.001 A 50.000,GrandariaMunicipi_DE 2.001 A 10.000,GrandariaMunicipi_FINS A 2.000,GrandariaMunicipi_MÉS DE 50.000 - RESTA CATALUNYA
0,100,674.941909,0,0,0,2,1,15,3,4,...,False,True,False,False,False,False,False,False,False,True
1,201,674.941909,0,0,0,2,1,16,3,4,...,False,True,False,False,False,False,False,False,False,True
2,202,674.941909,1,0,0,1,1,16,3,4,...,False,True,False,False,False,False,False,False,False,True
3,303,674.941909,0,0,0,1,1,16,3,5,...,False,True,False,False,False,False,False,False,False,True
4,501,674.941909,0,0,0,1,1,16,3,99,...,False,True,False,False,False,False,False,False,False,True


In [27]:
# A ver cuántos NaN tengo ahora que he hecho el ajuste de las columnas agrupadas:
nan_percentage = epcc23.isna().mean() * 100

# Crear un DataFrame con la información
nan_info = pd.DataFrame({
    'Column': nan_percentage.index,
    'NaN_Percentage': nan_percentage.values
})

# Mostrar el DataFrame ordenado por el porcentaje de NaNs
nan_info_sorted = nan_info.sort_values(by='NaN_Percentage', ascending=False)
print(nan_info_sorted)

                                                Column  NaN_Percentage
203                                             VJO13B       94.883608
507                                                CS6       92.507274
494                                                OR3       91.173618
184                                              VJO7D       90.809893
183                                              VJO7C       90.809893
..                                                 ...             ...
180                                            VJO7A#2        0.000000
179                                            VJO7A#1        0.000000
176                                            VJO6#99        0.000000
175                                            VJO6#95        0.000000
529  GrandariaMunicipi_MÉS DE 50.000 - RESTA CATALUNYA        0.000000

[530 rows x 2 columns]


In [28]:
nan_info_nonzero = nan_info_sorted[nan_info_sorted['NaN_Percentage'] > 0]
print(nan_info_nonzero)


         Column  NaN_Percentage
203      VJO13B       94.883608
507         CS6       92.507274
494         OR3       91.173618
184       VJO7D       90.809893
183       VJO7C       90.809893
..          ...             ...
51   CA2A#value        0.218235
69    CA6#value        0.193986
30    PP3#value        0.169738
42     R3#value        0.169738
20     D3#value        0.048497

[84 rows x 2 columns]


In [29]:
# Definir las columnas a verificar para NaN y rellenar con 99, 999 y 99999
columns_to_99 = ['I1', 'I4', "I3", "D1", "PP1", "R1", "CA1", "T3", "CA51", "IT2", "IT5", "VJO2", "VJO3#1", "VJO3", "VJO3#2", "VJO4#1", "VJO4#2", "VJO5", "VJ1", "VJ4", 
                 "VJ5", "VJO7", "VJO7B", "VJO8", "VJO9", "VJO10", "VJO12", "VJO13", "VJO13B", "M1", "M2", "CO1", "CO2", "CO10", "CN1", "CN1B", "CN10", "ES1", "ES2A", "ES10", 
                 "EX1", "EX10", "EX5", "EX10B", "EX11", "LL1", "LL1A", "LL3", "LL4", "LL6", "LL12", "LL13", "BI1", "LN6#8", "AS1", "LN6#7", "LN1", "LN3", "LN4", "LN5", "LN6#1", 
                 "LN6#2", "LN6#3", "LN6#4", "LN6#5", "LN6#6", "LN6#9", "Q10B", "OR2", "OR3", "G5", "OR4", "OR6", "OR7", "CS1", "CS2", "CS3", "CS6", "CS11", "CS12", "CS15A", 
                 "CS15", "CS21", "G2", "G2A"]

columns_to_999 = ["CO1A#2", "CO1A#1", "VJO11", "VJ1B", "CN1A", "G6", "G1", "LL13A", "LL2", "EX1A#1", "EX1A#2", "EX1A#3", "EX1A#4", "EX1A#5", "ES1A"]

columns_to_99999 = ["VJO7C"]

columns_to_3 = ["VJO6"]

# Contador de casillas modificadas
nan_to_99_modified_count = 0
nan_to_999_modified_count = 0
nan_to_99999_modified_count = 0
nan_to_3_modified_count = 0


# Rellenar con 99
for col in columns_to_99:
    if col in epcc23.columns:
        nan_count = epcc23[col].isna().sum()
        epcc23[col] = epcc23[col].fillna(99)
        nan_to_99_modified_count += nan_count

# Rellenar con 999
for col in columns_to_999:
    if col in epcc23.columns:
        nan_count = epcc23[col].isna().sum()
        epcc23[col] = epcc23[col].fillna(999)
        nan_to_999_modified_count += nan_count

# Rellenar con 99999
for col in columns_to_99999:
    if col in epcc23.columns:
        nan_count = epcc23[col].isna().sum()
        epcc23[col] = epcc23[col].fillna(99999)
        nan_to_99999_modified_count += nan_count

# Rellenar con 3
for col in columns_to_3:
    if col in epcc23.columns:
        nan_count = epcc23[col].isna().sum()
        epcc23[col] = epcc23[col].fillna(3)
        nan_to_3_modified_count += nan_count

# Mostrar el total de casillas modificadas
print(f"Total de casillas modificadas para NaN rellenadas con 99: {nan_to_99_modified_count}")
print(f"Total de casillas modificadas para NaN rellenadas con 999: {nan_to_999_modified_count}")
print(f"Total de casillas modificadas para NaN rellenadas con 99999: {nan_to_99999_modified_count}")
print(f"Total de casillas modificadas para NaN rellenadas con 3: {nan_to_3_modified_count}")

# Comprobar el porcentaje de NaNs por columna después de los cambios
nan_percentage = epcc23.isna().mean() * 100
nan_info = pd.DataFrame({
    'Column': nan_percentage.index,
    'NaN_Percentage': nan_percentage.values
})
nan_info_sorted = nan_info.sort_values(by='NaN_Percentage', ascending=False)
print(nan_info_sorted.head(20))

Total de casillas modificadas para NaN rellenadas con 99: 98925
Total de casillas modificadas para NaN rellenadas con 999: 31914
Total de casillas modificadas para NaN rellenadas con 99999: 3745
Total de casillas modificadas para NaN rellenadas con 3: 2704
         Column  NaN_Percentage
184       VJO7D       90.809893
487     LN7#4#3       81.595538
486     LN7#4#2       76.454898
484     LN7#4#0       72.672163
485     LN7#4#1       72.672163
488     LN7#4#9       27.376334
496    G5#value        2.546072
448   LN1#value        1.842871
445   AS2#value        1.818623
460   LN4#value        1.818623
458   LN3#value        1.697381
399  LL5A#value        1.624636
456   LN2#value        1.576140
197  VJO8#value        1.382153
83    CA5#value        1.236663
489   LN7#value        1.115422
116  VJO1#value        0.630456
177  VJO6#value        0.484966
51   CA2A#value        0.218235
69    CA6#value        0.193986


In [30]:
nan_info_nonzero = nan_info_sorted[nan_info_sorted['NaN_Percentage'] > 0]
print(nan_info_nonzero)

         Column  NaN_Percentage
184       VJO7D       90.809893
487     LN7#4#3       81.595538
486     LN7#4#2       76.454898
484     LN7#4#0       72.672163
485     LN7#4#1       72.672163
488     LN7#4#9       27.376334
496    G5#value        2.546072
448   LN1#value        1.842871
445   AS2#value        1.818623
460   LN4#value        1.818623
458   LN3#value        1.697381
399  LL5A#value        1.624636
456   LN2#value        1.576140
197  VJO8#value        1.382153
83    CA5#value        1.236663
489   LN7#value        1.115422
116  VJO1#value        0.630456
177  VJO6#value        0.484966
51   CA2A#value        0.218235
69    CA6#value        0.193986
30    PP3#value        0.169738
42     R3#value        0.169738
20     D3#value        0.048497


In [31]:
# 5. Manejo de strings únicos y celdas vacías
string_columns = epcc23.select_dtypes(include=['object']).columns

# Contador de casillas modificadas para strings
nan_to_na_modified_count = 0
dash_to_zero_modified_count = 0

for col in string_columns:
    # Contar y reemplazar NaNs por 'N/A'
    nan_count = epcc23[col].isna().sum()
    epcc23[col] = epcc23[col].fillna('N/A')
    nan_to_na_modified_count += nan_count

    # Contar y reemplazar '-' por '0'
    dash_count = (epcc23[col] == '-').sum()
    epcc23[col] = epcc23[col].replace('-', '0')
    dash_to_zero_modified_count += dash_count

# Mostrar el total de casillas modificadas para strings rellenadas con N/A y dashes reemplazadas con 0
print(f"Total de casillas modificadas para NaN rellenadas con N/A: {nan_to_na_modified_count}")
print(f"Total de casillas modificadas para '-' reemplazadas con 0: {dash_to_zero_modified_count}")


Total de casillas modificadas para NaN rellenadas con N/A: 766
Total de casillas modificadas para '-' reemplazadas con 0: 78690


In [32]:
nan_percentage = epcc23.isna().mean() * 100
nan_info = pd.DataFrame({
    'Column': nan_percentage.index,
    'NaN_Percentage': nan_percentage.values
})
nan_info_sorted = nan_info.sort_values(by='NaN_Percentage', ascending=False)
print(nan_info_sorted.head())  # Mostrar las 20 primeras columnas con más NaNs

      Column  NaN_Percentage
184    VJO7D       90.809893
487  LN7#4#3       81.595538
486  LN7#4#2       76.454898
485  LN7#4#1       72.672163
484  LN7#4#0       72.672163


In [33]:
# 1. Ahora llenar los NaNs restantes con 0
epcc23.fillna(0, inplace=True)

In [34]:
nan_percentage = epcc23.isna().mean() * 100
nan_info = pd.DataFrame({
    'Column': nan_percentage.index,
    'NaN_Percentage': nan_percentage.values
})
nan_info_sorted = nan_info.sort_values(by='NaN_Percentage', ascending=False)
print(nan_info_sorted.head())  # Mostrar las 20 primeras columnas con más NaNs

     Column  NaN_Percentage
0        ID             0.0
348   EX7#6             0.0
362    EX11             0.0
361   EX10B             0.0
360  EX9#99             0.0


In [35]:
epcc23

,ID,Elevació,Mètode,SEXE,EDAT,I1,I2EA,I2EB,AJ1,I4,...,AmbitTerritorial_COMARQUES GIRONINES,AmbitTerritorial_METROPOLITÀ,AmbitTerritorial_PENEDÈS,AmbitTerritorial_PONENT,AmbitTerritorial_TERRES DE L'EBRE,GrandariaMunicipi_BARCELONA CIUTAT,GrandariaMunicipi_DE 10.001 A 50.000,GrandariaMunicipi_DE 2.001 A 10.000,GrandariaMunicipi_FINS A 2.000,GrandariaMunicipi_MÉS DE 50.000 - RESTA CATALUNYA
0,100,674.941909,0,0,0,2,1,15,3,4,...,False,True,False,False,False,False,False,False,False,True
1,201,674.941909,0,0,0,2,1,16,3,4,...,False,True,False,False,False,False,False,False,False,True
2,202,674.941909,1,0,0,1,1,16,3,4,...,False,True,False,False,False,False,False,False,False,True
3,303,674.941909,0,0,0,1,1,16,3,5,...,False,True,False,False,False,False,False,False,False,True
4,501,674.941909,0,0,0,1,1,16,3,99,...,False,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4119,399701,3120.629630,0,1,4,2,2,59,3,3,...,False,False,True,False,False,False,True,False,False,False
4120,399902,3120.629630,0,1,4,1,2,57,3,4,...,False,False,True,False,False,False,True,False,False,False
4121,400100,3120.629630,1,1,4,1,2,73,6,1,...,False,False,True,False,False,False,True,False,False,False
4122,400103,3120.629630,0,1,4,2,2,77,3,2,...,False,False,True,False,False,False,False,False,False,True


In [36]:
conditional_questions = [
    {"column": "I1", "trigger": [1, 2], "dependents": ["I2EB", 
                                                       "AJ1", 
                                                       "I4", 
                                                       "I3"]},
    {"column": "D1", "trigger": [1], "dependents": ['D2#1', "D2#2", "D2#4", "D2#99", 
                                                    'D3#1', "D3#2", "D3#3", "D3#99"]},
    {"column": "PP1", "trigger": [1], "dependents": ['PP0#1', 'PP0#2', 'PP0#3', 'PP0#99', 
                                                     'PP3#1', 'PP3#2', 'PP3#3', 'PP3#99']},
    {"column": "R1", "trigger": [1], "dependents": ['R2#1', 'R2#2', 'R2#3', 'R2#4', 'R2#5', 'R2#99', 
                                                    'R3#1', 'R3#2', 'R3#3', 'R3#99']},
    {"column": "CA1", "trigger": [1, 2, 3, 4, 5, 6], "dependents": ['CA2A#1', 'CA2A#2', 'CA2A#3', 'CA2A#4', 'CA2A#5', 'CA2A#6', 'CA2A#99', 
                                                    'T4#2', 'T4#3', 'T4#4', 'T4#5', 'T4#6', 'T4#7', 'T4#8', 'T4#11', 'T4#99',
                                                     "T3", 
                                                     "CA6#1", "CA6#2", "CA6#3", "CA6#4", "CA6#5", "CA6#6", "CA6#99",
                                                     "CA4#1", "CA4#2", "CA4#3", "CA4#4", "CA4#99", 
                                                     "CA51", 
                                                     "CA5#1", "CA5#2", "CA5#3", "CA5#4", "CA5#5", "CA5#6", "CA5#99"]},
    {"column": "IT2", "trigger": [1], "dependents": ['IT5']},
    {"column": "VJO5", "trigger": [1, 2, 3], "dependents": ["VJ2#1", "VJ2#2", "VJ2#3", "VJ2#4", "VJ2#5", "VJ2#6", "VJ2#7", "VJ2#8", "VJ2#9", "VJ2#10", "VJ2#99"]},
    {"column": "VJO5", "trigger": [4, 5], "dependents": ["VJO1#1", "VJO1#2", "VJO1#3", "VJO1#4", "VJO1#5", "VJO1#6", "VJO1#7", "VJO1#8", "VJO1#9", "VJO1#10", "VJO1#11", "VJO1#99", 
                                                        "VJ1B", 
                                                        "VJ3#1", "VJ3#2", "VJ3#3", "VJ3#4", "VJ3#5", "VJ3#6", "VJ3#7", "VJ3#8", "VJ3#99", 
                                                        "VJ4", 
                                                        "VJ5", 
                                                        "VJ6#1", "VJ6#2", "VJ6#3", "VJ6#4", "VJ6#5", "VJ6#6", "VJ6#7", "VJ6#99", 
                                                        "VJ7#1", "VJ7#2", "VJ7#3", "VJ7#4", "VJ7#5", "VJ7#6", "VJ7#7", "VJ7#8", "VJ7#9", "VJ7#10", "VJ7#11", "VJ7#99", 
                                                        "VJO6" , "VJO6#1", "VJO6#2", "VJO6#3", "VJO6#4", "VJO6#5", "VJO6#6", "VJO6#7", "VJO6#95", "VJO6#99", # ????
                                                        "VJO7"]}, # VJO5 DE EXCEL / CSV PARECE SER P18 EN EL PDF
    {"column": "VJO7", "trigger": [1], "dependents": ['VJO7A#1', 'VJO7A#2', 'VJO7A#99', 
                                                      'VJO7B']},
    {"column": "VJO8", "trigger": [1], "dependents": ['VJO8#1', 'VJO8#2', 'VJO8#3', 'VJO8#4', 'VJO8#5', 'VJO8#6', 'VJO8#7', 'VJO8#8', "VJO8#9", "VJO8#95", "VJO8#99"]},
    {"column": "VJO7B", "trigger": [1], "dependents": ['VJO7C', 
                                                       'VJO7D']},
    {"column": "I3", "condition": lambda x: x != 0, "dependents": ['VJO9', 
                                                                   'VJO10', 
                                                                   'VJO11', 
                                                                   'VJO12', 
                                                                   'VJO13']},
    {"column": "VJO13", "trigger": [1], "dependents": ['VJO13B']},
    {"column": "M1", "trigger": [1], "dependents": ['M2', 
                                                    "M3#1", "M3#2", "M3#3", "M3#4", "M3#5", "M3#6", "M3#7", "M3#99", 
                                                    "M4#1", "M4#7", "M4#9", "M4#10", "M4#11", "M4#13", "M4#18", "M4#19", "M4#99", 
                                                    "M5#1", "M5#2", "M5#3", "M5#4", "M5#5", "M5#6", "M5#7", "M5#8", "M5#9", "M5#99"]},
    {"column": "M1", "trigger": [0], "dependents": ['M7#1', 'M7#2', 'M7#3', 'M7#4', 'M7#5', 'M7#6', 'M7#7', 'M7#8', 'M7#9', 'M7#10', 'M7#11', 'M7#99']},
    {"column": "CO1", "trigger": [1], "dependents": ['CO1A#1', "CO1A#2",
                                                     'CO3#1', 'CO3#2', 'CO3#3', 'CO3#4', 'CO3#5', 'CO3#6', 'CO3#7', 'CO3#99', 
                                                     "CO9#1", "CO9#7", "CO9#9", "CO9#10", "CO9#11", "CO9#13", "CO9#18", "CO9#19", "CO9#99", 
                                                     "CO10"]},
    {"column": "CO1A#2", "condition": lambda x: x > 0, "dependents": ['CO2']},
    {"column": "CO1", "trigger": [0], "dependents": ['CO7#1', 'CO7#2', 'CO7#3', 'CO7#4', 'CO7#5', 'CO7#6', 'CO7#7', 'CO7#8', 'CO7#9', 'CO7#10', 'CO7#99']},
    {"column": "CN1", "trigger": [1], "dependents": ['CN1A', 
                                                     "CN2#1", "CN2#2", "CN2#3", "CN2#4", "CN2#5", "CN2#6", "CN2#99", 
                                                     "CN1B", 
                                                     "CN5#1", "CN5#2", "CN5#3", "CN5#4", "CN5#5", "CN5#6", "CN5#9", "CN5#10", "CN5#11", "CN5#12", "CN5#13", "CN5#99", 
                                                     "CN10"]},
    {"column": "CN1", "trigger": [0], "dependents": ['CN6#1', 'CN6#2', 'CN6#3', 'CN6#4', 'CN6#5', 'CN6#6', 'CN6#7', 'CN6#8', 'CN6#9', 'CN6#88', 'CN6#99']},
    {"column": "ES1", "trigger": [1], "dependents": ['ES1A', 
                                                     "ES2A", 
                                                     "ES3#1", "ES3#2", "ES3#3", "ES3#4", "ES3#5", "ES3#6", "ES3#7", "ES3#99", 
                                                     "ES10"]},
    {"column": "ES1", "trigger": [0], "dependents": ['ES6#1', 'ES6#2', 'ES6#3', 'ES6#4', 'ES6#5', 'ES6#6', 'ES6#7', 'ES6#8', 'ES6#9', 'ES6#88', 'ES6#99']},
    {"column": "EX1", "trigger": [1], "dependents": ['EX1A#1', 'EX1A#2', 'EX1A#3', 'EX1A#4', 'EX1A#5', 
                                                     "EX10", 
                                                     "EX5"]},
    {"column": "EX1", "trigger": [0], "dependents": ['EX7#1', 'EX7#2', 'EX7#3', 'EX7#4', 'EX7#5', 'EX7#6', 'EX7#7', 'EX7#8', 'EX7#9', 'EX7#88', 'EX7#99']},
    {"column": "EX1A#1", "condition": lambda x: x > 0, "dependents": ['EX9#1', 'EX9#2', 'EX9#3', 'EX9#4', 'EX9#5', 'EX9#6', 'EX9#99',
                                                                      "EX10B", 
                                                                      "EX11", 
                                                                      "EX13#1", "EX13#1", "EX13#1", "EX13#1", "EX13#1", "EX13#1", "EX13#1", "EX13#1", "EX13#99"]},
    {"column": "LL1", "trigger": [1], "dependents": ['LL1A', 
                                                     "LL2", 
                                                     "LL3", 
                                                     "LL4", 
                                                     "LL5#2", "LL5#3", "LL5#4", "LL5#5", "LL5#6", "LL5#7", "LL5#8", "LL5#9", "LL5#10", "LL5#11", "LL5#99", 
                                                     "LL5A#1", "LL5A#2", "LL5A#3", "LL5A#4", "LL5A#5", "LL5A#6", "LL5A#7", "LL5A#8", "LL5A#9", "LL5A#10", "LL5A#99", 
                                                     "LL6", 
                                                     "LL12"]},
    {"column": "LL1", "trigger": [0], "dependents": ["LL11#11", "LL11#12", "LL11#2", "LL11#3", "LL11#4", "LL11#7", "LL11#13", "LL11#9", "LL11#16", "LL11#99",]},
    {"column": "LL1", "condition": lambda x: (x > 0) & (x < 99), "dependents": ["LL13"]},
    {"column": "LL13", "trigger": [1], "dependents": ["LL13A"]},
    {"column": "BI1", "trigger": [1, 2, 3, 4, 5, 6], "dependents": ["BI2#1", "BI2#2", "BI2#3", "BI2#4", "BI2#99"]},
    {"column": "BI1", "condition": lambda x: (x > 0) & (x < 7), "dependents": ["BI3#1", "BI3#2", "BI3#3", "BI3#5", "BI3#6", "BI3#7", "BI3#8", "BI3#10", "BI3#99", 
                                                                               "LN6#8"]},
    {"column": "AS1", "trigger": [1, 2], "dependents": ["AS2#1", "AS2#2", "AS2#3", "AS2#4", "AS2#5", "AS2#6", "AS2#7", "AS2#8", "AS2#9", "AS2#10", "AS2#11", 
                                                        "AS2#16", "AS2#17", "AS2#99",
                                                        "LN6#7"]},
    {"column": "OR2", "trigger": [3], "dependents": ["G6"]},
    {"column": "OR2", "trigger": [2], "dependents": ["OR3"]},
    {"column": "CS1", "trigger": [1], "dependents": ["CS2"]},
    {"column": "CS2", "trigger": [1], "dependents": ["CS3"]},
    {"column": "CS1", "trigger": [2], "dependents": ["CS6"]},
    {"column": "CS1", "trigger": [3], "dependents": ["CS11"]},
    {"column": "CS1", "trigger": [4, 5], "dependents": ["CS12"]},
    {"column": "CS15A", "condition": lambda x: x != 1, "dependents": ["CS15", "CS21"]},
    {"column": "I4", "trigger": [3, 4, 5, 6], "dependents": ["G2"]},
    {"column": "I4", "trigger": [1, 2], "dependents": ["G2A"]},
]

# Crear un diccionario para las nuevas columnas
new_columns = {}

for q in conditional_questions:
    column = q["column"]
    if 'trigger' in q:
        triggers = q['trigger']
        for dependent in q['dependents']:
            if dependent in epcc23.columns:
                new_column_name = f'answered_{dependent}'
                new_columns[new_column_name] = np.where(epcc23[column].isin(triggers), epcc23[dependent], np.nan)
    elif 'condition' in q:
        cond_function = q['condition']
        for dependent in q['dependents']:
            if dependent in epcc23.columns:
                new_column_name = f'answered_{dependent}'
                new_columns[new_column_name] = np.where(cond_function(epcc23[column]), epcc23[dependent], np.nan)

# Añadir todas las nuevas columnas al DataFrame de una vez
new_columns_df = pd.DataFrame(new_columns, index=epcc23.index)
epcc23 = pd.concat([epcc23, new_columns_df], axis=1)


In [37]:
# 7. Marcar similitud o paralelismo entre columnas
# Definir los motivos comunes
motivos_comunes = {
    'Manca de temps': ['VJ2#1', 'M7#1', "CO7#1", "CN6#1", "ES6#1", "EX7#1", "LL11#11"],
    'Responsabilitats familiars': ['VJ2#2', 'M7#2', "CO7#2", "CN6#2", "ES6#2", "EX7#2", "LL11#12"],
    'Manca d’interès': ['VJ2#3', 'M7#4', "CO7#4", "CN6#4", "ES6#4", "EX7#4", "LL11#3"],
    'Manca d’informació': ['VJ2#4', 'M7#5', "CO7#5", "ES6#5", "EX7#5", "LL11#4"],
    'Problemes de salut': ['VJ2#5', 'M7#7', "CO7#6", "CN6#5", "ES6#6", "EX7#6", "LL11#7"],
    'No tenir amb qui': ['VJ2#6', 'M7#8', "CO7#7", "CN6#6", "ES6#7", "EX7#7", "LL11#13"],
    'Manca d’oferta': ['VJ2#7', 'M7#9', "CO7#8", "CN6#7", "ES6#8", "EX7#8", "LL11#9"],
    'Preu': ['VJ2#8', 'M7#3', "CO7#3", "CN6#3", "ES6#3", "EX7#3", "LL11#2"],
    'Dificultats d’accessibilitat': ['VJ2#9', 'M7#10', "CO7#9", "CN6#9", "ES6#9", "EX7#9"],
    'Altres': ['VJ2#10', 'M7#11', "CO7#10", "CN6#88", "ES6#88", "EX7#88", "LL11#16"],
    "NS/NC": ["VJ2#99", "M7#00", "CO7#99", "CN6#99", "ES6#99", "EX7#99", "LL11#99"],
    "Baixa qualitat": ["M7#6"],
    "Ho faig per una altra via": ["CN6#8"]
}

# Crear nuevas columnas para cada motivo común y inicializarlas en 0
for motivo in motivos_comunes:
    epcc23[f'{motivo}_total'] = 0

# Actualizar las nuevas columnas basándose en los motivos existentes
for motivo, cols in motivos_comunes.items():
    for col in cols:
        if col in epcc23.columns:
            epcc23[f'{motivo}_total'] += epcc23[col]


##### Al menos de momento voy a dejar identificadas y apartadas las columnas con string que tienen valores no estructurados

In [38]:
# Identificar las columnas que terminan en #value
value_columns = [col for col in epcc23.columns if col.endswith('#value')]

# Crear un DataFrame separado para estas columnas
value_df = epcc23[value_columns]

# Eliminar estas columnas del DataFrame principal
epcc23 = epcc23.drop(columns=value_columns)

In [39]:
# Ver los valores únicos para cada una de las columnas #value
for col in value_df.columns:
    print(f"Valores únicos para la columna {col}:")
    print(value_df[col].unique())
    print("\n")


Valores únicos para la columna D3#value:
['0' '-ÁNGLES' 'INGLÉS' 'INGLÉS,JAPONÉS' 'N/A' 'ANGLÈS' 'ANGLES' '-INGLES'
 'ARABE' 'INGLES' 'ALEMANY, ANGLÈS' 'ALEMANY' 'ANGELS' 'HOLANDES, INGLES'
 'ÁRABE' 'ITALIANO E INGLÉS' 'ENGLISH' 'PORTUGUÉS E INGLES'
 'ANGLÈS, ITALIÀ' 'VASC, ANGLES' 'ANGLÉS , FRANCÉS I ITALIÀ'
 'INGLES, ALEMAN' 'FRANCES INGLES' 'URDU' 'ITALIANO' 'ANGLÉS' 'FRANCÉS'
 '-ANGLES' 'ALEMANY I INGLÉS' 'EN INGLES BBC NEWS' 'FRANCÉS, ANGLÈS'
 'FRANCES' 'ALEMÁN, INGLÉS' 'GALLEC' 'INGLÉS Y FRANCÉS' 'RUSO'
 '-INGLES Y FRANCES' 'PORTUGUÉS' 'BÚLGARO' 'ALEMÁN'
 'ITALIÀ, ANGLÈS, FRANCÈS, ALEMANY' 'CHINO MANDARÍN, PEQUINES Y RUSO'
 'FRANCÈS' 'UKRANIANO, RUSO' 'ITALIANO - INGLES' 'SUECO'
 'ALEMANY FRANCÈS ANGLÈS' 'GALLEGO' 'RUMANO' 'ÁRABES' 'ANGLES I FRANCES'
 'ÁRABE ,INGLES' 'PORTUGUES' 'FRANCÉS Y INGLÉS' 'ANGLÉS I POLONÉS']


Valores únicos para la columna PP3#value:
['0' 'INGLES' 'ANGLÉS' 'INGLÉS' 'ANGLÈS' 'ANGLES' 'INGLES, GALLEGO' 'N/A'
 'FRANCES' 'ANGLE' '-INGLES' 'FRACÈS I ANGLES' 